In [8]:
import pandas as pd

## Generate ratings.csv

Convert four behaviors into explicit ratings:

['click', 'collect', 'cart', 'alipay'] → [1.25, 2.5, 3.75, 5]

In [9]:
# Read raw data
df = pd.read_csv("./raw/(sample)sam_tianchi_2014002_rec_tmall_log.csv", encoding='GB2312')
df.groupby('action').count()

,item_id,user_id,vtime
action,,,
alipay,7,7,7
cart,358,358,358
click,4474,4474,4474
collect,5,5,5


In [10]:
# Convert four behaviors into explicit ratings
df.action = df.action.replace(
    ['click', 'collect', 'cart', 'alipay'], 
    [1.25, 2.5, 3.75, 5])
df.groupby('action').count()

,item_id,user_id,vtime
action,,,
1.25,4474,4474,4474
2.50,5,5,5
3.75,358,358,358
5.00,7,7,7


In [11]:
# Save to ratings.csv
df.to_csv('ratings.csv', index=False)

## Generate kg.txt